In [1]:
import pandas as pd

# Загружаем таблицы
main_df = pd.read_csv('data/dish.csv')  
classifier_df = pd.read_csv('data/ingredients.csv')  



In [2]:
# Создаем словарь для замены id -> ingr
id_to_ingr = dict(zip(classifier_df['id'], classifier_df['ingr']))
print(list(id_to_ingr.items())[:10])


[(1, 'cottage cheese'), (2, 'strawberries'), (3, 'garden salad'), (4, 'bacon'), (5, 'potatoes'), (6, 'caesar salad'), (7, 'cauliflower'), (8, 'scrambled eggs'), (9, 'wild rice'), (10, 'steak')]


In [3]:
# 1. Меняем split для dish_id = "dish_1562085672" с test на train
# есть блюда, которые есть в выборке тест, но их нет в выборке трейн
# bread, tuna salad
print(f"\n1. Изменяем split для dish_1562085672...")
dish_ids_to_change = ['dish_1562085672', 'dish_1568401167']
mask = main_df['dish_id'].isin(dish_ids_to_change)
changed_count = mask.sum()
main_df.loc[mask, 'split'] = 'train'
print(f"   ✓ Split изменен на 'train' для {changed_count} записей")
    



1. Изменяем split для dish_1562085672...
   ✓ Split изменен на 'train' для 2 записей


In [4]:
# 2. Удаляем dish_id = "dish_1560974769"
# исключаем блюда содержащие выброс (неадекватное значение по весу 3 051)
print(f"\n2. Удаляем dish_1560974769...")
initial_count = len(main_df)
main_df = main_df[main_df['dish_id'] != 'dish_1560974769']
removed_count = initial_count - len(main_df)
print(f"   Удалено записей: {removed_count}")


2. Удаляем dish_1560974769...
   Удалено записей: 1


In [5]:
# 3. Удаляем записи с запрещенными кодами в ingredients
# deprecated /устаревшие
# 423	plate only
# 231	deprecated
# 453	deprecated 6/15 (test/train)
# 458	deprecated 2/8 (test/train)
# 470	deprecated


banned_codes = ['423', '231', '453', '458', '470']
print(f"\n3. Удаляем записи с запрещенными кодами: {banned_codes}")

def contains_banned_codes(ingredients_str):
    """Проверяет, содержит ли строка запрещенные коды"""
    if pd.isna(ingredients_str):
        return False
    
    ingredients_list = str(ingredients_str).split(';')
    for ingredient in ingredients_list:
        ingredient = ingredient.strip()
        # Проверяем наличие запрещенных кодов в составе ингредиента
        for banned_code in banned_codes:
            if banned_code in ingredient:
                return True
    return False

# Применяем фильтрацию
initial_count_before_banned = len(main_df)
banned_mask = main_df['ingredients'].apply(contains_banned_codes)
main_df = main_df[~banned_mask]
banned_removed_count = initial_count_before_banned - len(main_df)
print(f"   Удалено записей с запрещенными кодами: {banned_removed_count}")


3. Удаляем записи с запрещенными кодами: ['423', '231', '453', '458', '470']
   Удалено записей с запрещенными кодами: 33


In [6]:
def process_ingredients(ingredients_str):
    """Обрабатывает строку с ингредиентами"""
    if pd.isna(ingredients_str):
        return ""
    
    ingredients_list = str(ingredients_str).split(';')
    processed_ingredients = []
    
    for ingredient in ingredients_list:
        ingredient = ingredient.strip()
        
        # Пропускаем ingr_0000000291 (Соль)
        if ingredient == 'ingr_0000000291':
            continue
        
        if ingredient.startswith('ingr_'):
            # Преобразуем "ingr_0000000122" в число 122
            ingredient_id = int(ingredient[5:])
            
            # Заменяем на значение из классификатора
            if ingredient_id in id_to_ingr:
                processed_ingredients.append(id_to_ingr[ingredient_id])
            else:
                processed_ingredients.append(ingredient)
        else:
            processed_ingredients.append(ingredient)
    
    # Объединяем обратно в строку с разделителем ";"
    return ";".join(processed_ingredients)

In [7]:
# Применяем функцию к столбцу ingredients
main_df['processed_ingredients'] = main_df['ingredients'].apply(process_ingredients)



In [8]:
print(main_df[:10])

            dish_id  total_calories  total_mass  \
0   dish_1561662216      300.794281       193.0   
1   dish_1561662054      419.438782       292.0   
2   dish_1562008979      382.936646       290.0   
3   dish_1560455030       20.590000       103.0   
5   dish_1565640549       45.482903       139.0   
6   dish_1563207364      309.269989       271.0   
7   dish_1561575474      120.058434       183.0   
8   dish_1563216717      246.007996       332.0   
9   dish_1565972591      195.199997       122.0   
10  dish_1568649312       44.459999        78.0   

                                          ingredients  split  \
0   ingr_0000000508;ingr_0000000122;ingr_000000002...   test   
1   ingr_0000000312;ingr_0000000026;ingr_000000002...  train   
2   ingr_0000000448;ingr_0000000520;ingr_000000046...   test   
3     ingr_0000000471;ingr_0000000031;ingr_0000000347  train   
5   ingr_0000000032;ingr_0000000523;ingr_000000016...  train   
6   ingr_0000000008;ingr_0000000192;ingr_000000044... 

In [9]:
# Сохраняем результат
main_df.to_csv('data/dish_processed.csv', index=False)

train_df = main_df[main_df['split'] == 'train']
test_df = main_df[main_df['split'] == 'test']

# Сохраняем в отдельные файлы
train_df.to_csv('data/train_data.csv', index=False)
test_df.to_csv('data/test_data.csv', index=False)

# Выводим статистику

print(f"Общее количество строк: {len(main_df)}")
print(f"Количество строк в train: {len(train_df)}")
print(f"Количество строк в test: {len(test_df)}")

print("Обработка завершена!")

Общее количество строк: 3228
Количество строк в train: 2732
Количество строк в test: 496
Обработка завершена!
